# Copy all HAT-P-67 observations to UTBox

## So that Quang can run an RV planet search on it

In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import pyarrow.feather as feather
import os

In [3]:
df = feather.read_feather('../../data/HPF/HAT_P_67_log.feather')

In [4]:
#! mkdir /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_glks

In [5]:
n_spec = len(df)
n_spec

152

In [6]:
for i in range(n_spec):
    fn = df.full_path.iloc[i]
    cmd = f'cp {fn} /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_glks/'
    #return_status = os.system(cmd)

In [7]:
! du -hs /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_glks/

409M	/Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_glks/


In [8]:
! ls /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_glks/ | wc -l

     152


## Do the same for HPF team reduction

Quang prefers the HPF team reduction files, so let's do the same thing for those!

In [9]:
import numpy as np
import os
import glob
from astropy.io import fits
import warnings
import matplotlib.pyplot as plt
import astropy.units as u
from muler.hpf import HPFSpectrum
import seaborn as sns

from specutils.fitting import fit_continuum
from astropy.modeling import models

from astropy.time import Time
from tqdm.notebook import tqdm

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('paper', font_scale=1.2)

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore", message="Model is linear in parameters")
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="frame.append")

In [10]:
def get_metadata_from_fits(filenames, dynamic_only=True):
    df_out = pd.DataFrame()
    for file in tqdm(filenames):
        hdu = fits.open(file)
        hdr = hdu[0].header
        
        # Remove duplicate keys like `HISTORY`
        columns = np.array(list(hdr.keys()))
        _, indices, counts = np.unique(columns, return_index=True, return_counts=True)
        columns = columns[np.isin(columns, columns[indices[counts==1]])]
        df = pd.Series({col:hdr[col] for col in columns }).to_frame().T
        df['file']=file
        df_out = df_out.append(df, ignore_index=True)
        hdu.close()
    
    # Only keep columns that change in time
    if dynamic_only:
        dynamic_columns = df_out.apply(lambda x: len(pd.unique(x))) != 1
        df_out = df_out[dynamic_columns[dynamic_columns].index.values]
        
    return df_out

In [11]:
tacc_path = '/Volumes/pecos/HPF/TACC'

In [12]:
exosphere_program_ids = ['UT19-2-013', 'UT19-3-009',  'UT20-1-012', 
                         'UT20-2-011', 'UT20-3-009', 'UT21-1-010', 'UT22-2-019']

In [13]:
tacc_PSU_files = []
for prog_id in exosphere_program_ids:
    print(prog_id)
    pattern = tacc_path + '/{}/Slope*.optimal.fits'.format(prog_id)
    filenames = glob.glob(pattern, recursive=True)
    tacc_PSU_files += filenames

UT19-2-013
UT19-3-009
UT20-1-012
UT20-2-011
UT20-3-009
UT21-1-010
UT22-2-019


In [14]:
len(tacc_PSU_files)

1099

In [15]:
df_raw = get_metadata_from_fits(tacc_PSU_files)

  0%|          | 0/1099 [00:00<?, ?it/s]

In [16]:
df_raw['full_path'] = tacc_PSU_files

In [17]:
target_mask = df_raw.OBJECT.str.contains('67')
df = df_raw[target_mask]

In [18]:
df

,DATE,OBJECT,OBSERVER,DATE-OBS,CAL-OBJ,QTIME,QRA,QDEC,QPMRA,QPMDEC,...,JD_FW20,JD_FW23,JD_FW22,JD_FW25,JD_FW24,JD_FW27,JD_FW26,CHECKSUM,file,full_path
317,2020-04-27T06:28:47,HAT-P-67b_OUT_1_before1,CFB,2020-04-27T06:29:11.640106,dark,1587968927.824912791,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2458966.772049,2458966.772051,2458966.772055,2458966.772048,2458966.772048,2458966.772048,2458966.772049,34hjA4fi54fiA4fi,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...
318,2020-04-27T06:34:37,HAT-P-67b_OUT_1_before1,CFB,2020-04-27T06:35:03.305245,dark,1587969277.067140788,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2458966.776106,2458966.776102,2458966.776099,2458966.776107,2458966.776103,2458966.776106,2458966.776104,CAmaF4kWCAkaC3kU,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...
319,2020-04-27T06:40:29,HAT-P-67b_OUT_1_before1,CFB,2020-04-27T06:40:55.338979,dark,1587969629.827530023,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2458966.780232,2458966.780233,2458966.780232,2458966.780237,2458966.780236,2458966.780238,2458966.78024,Z22nd20kZ20kd20k,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...
320,2020-04-27T06:46:19,HAT-P-67b_OUT_1_before1,CFB,2020-04-27T06:46:47.563456,dark,1587969979.777269181,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2458966.784274,2458966.784277,2458966.784276,2458966.784276,2458966.784274,2458966.784275,2458966.784277,kD9BkA99kA9AkA97,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...
321,2020-04-28T06:10:02,HAT-P-67b_IN_1,CFB,2020-04-28T06:10:31.377897,dark,1588054201.814175072,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2458967.758759,2458967.758772,2458967.75877,2458967.758773,2458967.758765,2458967.758782,2458967.758778,5NEf6N9f5NEf5N9f,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...,/Volumes/pecos/HPF/TACC/UT20-2-011/Slope-20200...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,2021-02-01T12:35:45,hat-p-67b,RA,2021-02-01T12:36:08.756471,dark,1612182944.828903215,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2459247.028011,2459247.028013,2459247.02801,2459247.028015,2459247.028015,2459247.028016,2459247.028014,eRZghPZeePZeePZe,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...
1089,2021-02-24T10:55:11,hat-p-67b,RA,2021-02-24T10:55:40.061376,dark,1614164110.981470946,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2459269.959637,2459269.959637,2459269.959637,2459269.959637,2459269.959637,2459269.959637,2459269.959637,1gCZ4f9Z1fCZ1f9Z,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...
1090,2021-02-24T11:04:14,hat-p-67b,RA,2021-02-24T11:04:43.757994,dark,1614164653.918649685,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2459269.964833,2459269.964824,2459269.964837,2459269.964825,2459269.96483,2459269.964834,2459269.964826,QEjhS9jfQEjfQ9jf,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...
1097,2021-02-26T10:28:01,hat-p-67b,RA,2021-02-26T10:28:30.846267,LFC-shut,1614335281.022101646,17:06:26.56,+44:46:37.06,9.436,-18.185,...,2459271.9397,2459271.93969,2459271.939692,2459271.939688,2459271.939688,2459271.939684,2459271.939686,JVjaJShVJShaJShU,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...,/Volumes/pecos/HPF/TACC/UT21-1-010/Slope-20210...


In [19]:
df.OBJECT.value_counts()

HAT-P-67b_IN_1             14
HAT-P-67b_IN_2             14
HAT-P-67b_IN_3              9
HAT-P-67b_OUT_same3         9
hat-p-67b                   8
HAT-P-67b_OUT_1_before1     4
HAT-P-67b_OUT_1_after1      4
HAT-P-67b_OUT_2_before2     4
HAT-P-67b_OUT_2_before1     4
HAT-P-67b_OUT_2_after1      4
HAT-P-67b_OUT_2_after2      4
HAT-P-67b_OUT_3_before2     4
HAT-P-67b_OUT_3_before1     4
HAT-P-67b_OUT_3_after1      4
HAT-P-67b_OUT_3_after2      4
Hat-P-67                    4
Name: OBJECT, dtype: int64

In [20]:
target_name = 'HAT-P-67b'
orbital_period = 4.81010250
t_c = 2455961.38467
t_c_btjd = t_c - 2457000.0 #neg value means discovered before TESS
t_14_hours = 6.989 # transit duration in hours
t_14_days = t_14_hours/24.0
t_14_orb_phase = t_14_days / orbital_period

In [21]:
df['JD'] = df['DATE-OBS'].apply(lambda x: Time(x).jd)
df['BTJD'] = df.JD -2457000.0

df['phase'] = np.mod(df.BTJD-t_c_btjd+(orbital_period*0.5),orbital_period)-0.5*orbital_period
df['normalized_phase']=(df['phase']/orbital_period)
df['in_transit_flag']=np.abs(df['normalized_phase']) < t_14_orb_phase/2.0

In [22]:
hpf_t0 = Time(['2017-11-29T17:00:00.0']) # 17h UTC is ~noon in Fort Davis
JD_values = df['JD'].values
night_number = np.floor(JD_values - hpf_t0.jd).astype(int)
time_from_local_midnight = np.round(np.mod(JD_values - hpf_t0.jd, 1)*24.0 - 12.0, decimals=2)
df.insert(0, 'night_number', night_number)
nightly_timestamp = (hpf_t0 + df.night_number*u.day).iso
df.insert(1, 'night_of', [string[0:10] for string in nightly_timestamp])
df.insert(2, 'local_hour', time_from_local_midnight) # Negative means time until local midnight

Compare to Goldilocks

In [23]:
df_goldi = feather.read_feather('../../data/HPF/HAT_P_67_log.feather')

In [24]:
df.night_of.value_counts(sort=False).sort_index()

2020-04-26     4
2020-04-27    14
2020-04-28     4
2020-05-19     4
2020-05-20     4
2020-05-21    14
2020-05-22     4
2020-05-23     4
2020-06-12     4
2020-06-13     4
2020-06-14    18
2020-06-15     4
2020-06-17     4
2020-07-21     2
2020-07-31     2
2021-01-30     2
2021-01-31     2
2021-02-23     2
2021-02-25     2
Name: night_of, dtype: int64

In [25]:
df['QPROG'].value_counts()

UT20-2-011    90
UT21-1-010     8
Name: QPROG, dtype: int64

In [26]:
df_goldi['QPROG'].value_counts()

UT20-2-011    87
UT22-2-019    54
UT21-1-010    11
Name: QPROG, dtype: int64

In [27]:
df_goldi.night_of.value_counts(sort=False).sort_index()

2020-04-26     4
2020-04-27    12
2020-04-28     4
2020-05-19     4
2020-05-20     4
2020-05-21    14
2020-05-22     4
2020-05-23     4
2020-06-12     3
2020-06-13     4
2020-06-14    18
2020-06-15     4
2020-06-17     4
2020-07-21     2
2020-07-31     2
2021-01-30     2
2021-01-31     2
2021-02-23     1
2021-02-25     2
2021-03-03     2
2021-03-30     2
2022-04-27     4
2022-04-28    14
2022-04-29     4
2022-04-30     4
2022-05-01     4
2022-06-19     3
2022-06-21     6
2022-06-22     3
2022-06-25     1
2022-06-30     1
2022-07-09     1
2022-07-10     1
2022-07-12     3
2022-07-14     1
2022-07-15     1
2022-07-19     1
2022-07-28     1
2022-07-29     1
Name: night_of, dtype: int64

In [31]:
len(df.full_path.values)

98

In [32]:
n_psu = len(df.full_path.values)

In [33]:
! mkdir /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_psu/

In [36]:
for i in range(n_psu):
    fn = df.full_path.iloc[i]
    cmd = f'cp {fn} /Users/mag3842/GitHub/disperse/data/HPF/HAT_P_67_psu/'
    #print(cmd)
    #return_status = os.system(cmd)